In [1]:
pip install pandas numpy matplotlib tensorflow


     |████████████████████████████████| 601.3MB 21kB/s  eta 0:00:0111   |█████                           | 94.3MB 12.9MB/s eta 0:00:401MB 13.4MB/s eta 0:00:35     |███████████████████████▍        | 439.4MB 35.2MB/s eta 0:00:05     |█████████████████████████████▌  | 554.5MB 21.9MB/s eta 0:00:03
     |████████████████████████████████| 5.8MB 103.1MB/s eta 0:00:01
     |████████████████████████████████| 24.5MB 98.2MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 74.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 1.5MB/s  eta 0:00:01
     |████████████████████████████████| 5.1MB 48.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 1.3MB/s  eta 0:00:01
     |████████████████████████████████| 5.4MB 84.9MB/s eta 0:00:01            | 1.0MB 84.9MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 88.6MB/s eta 0:00:01
     |████████████████████████████████| 296kB 93.9MB/s eta 0:00:01
     |████████████████████████████████| 2.2MB 30.7MB/s eta 0:0

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


In [3]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"FACT_CUSTOMER_ORDERS"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [5]:
df = df[['PRODUCT_ID','ORDER_DATE','ORDER_QTY']]

In [8]:
df['ORDER_DATE']=pd.to_datetime(df['ORDER_DATE'])

In [9]:
df = df.sort_values('ORDER_DATE')

In [10]:
# Normalize the ORDER_QTY
scaler = MinMaxScaler(feature_range=(0, 1))
df['ORDER_QTY'] = scaler.fit_transform(df[['ORDER_QTY']])

# Create a new DataFrame with only the ORDER_QTY column
data = df[['ORDER_QTY']].values

# Split into train and test sets
train_size = int(len(data) * 0.67)
test_size = len(data) - train_size
train, test = data[0:train_size], data[train_size:len(data)]

# Convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# Reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
151125/151125 - 114s - 752us/step - loss: 0.0930
Epoch 2/100
151125/151125 - 113s - 746us/step - loss: 0.0926
Epoch 3/100
151125/151125 - 113s - 747us/step - loss: 0.0926
Epoch 4/100
151125/151125 - 113s - 746us/step - loss: 0.0925
Epoch 5/100
151125/151125 - 113s - 748us/step - loss: 0.0925
Epoch 6/100
151125/151125 - 113s - 746us/step - loss: 0.0925
Epoch 7/100
151125/151125 - 113s - 746us/step - loss: 0.0925
Epoch 8/100
151125/151125 - 113s - 746us/step - loss: 0.0925
Epoch 9/100
151125/151125 - 113s - 748us/step - loss: 0.0925
Epoch 10/100
151125/151125 - 113s - 745us/step - loss: 0.0925
Epoch 11/100
151125/151125 - 113s - 747us/step - loss: 0.0925
Epoch 12/100
151125/151125 - 113s - 748us/step - loss: 0.0925
Epoch 13/100
151125/151125 - 113s - 747us/step - loss: 0.0925
Epoch 14/100
151125/151125 - 112s - 744us/step - loss: 0.0925
Epoch 15/100
151125/151125 - 113s - 747us/step - loss: 0.0925
Epoch 16/100
151125/151125 - 112s - 742us/step - loss: 0.0925
Epoch 17/100
1511